King County, Washington

Here we will conduct exploratory data analysis on housing sale data in King County Washington from June 2021 to June 2022. The goal is to identify the best areas in the county to purchase, renovate, and resell homes. We will look at how to renovate later.

In [1]:
# Suppress future and deprecation warnings
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = DeprecationWarning)

# Standard Packages
import pandas as pd
import numpy as np
import datetime

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt

# Scipy Stats
import scipy.stats as stats 

# Statsmodel Api
import statsmodels.api as sm
from statsmodels.formula.api import ols

# SKLearn Modules
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

# Location visualization
import folium
from folium.plugins import MarkerCluster
import streamlit as st
from streamlit_folium import folium_static

In [2]:
# Pull csv into DataFrame, dataset provided, from https://data.kingcounty.gov/
df = pd.read_csv('data/kc_house_data.csv')

We will get rid of records outside of the County.

In [3]:
#location mask, will filter by zipcode instead
#king_county_loc_mask = (df['lat'] >= 47.1559) & (df['lat'] <= 47.8203) & (df['long'] >= -122.5428) & (df['long'] <= -121.7867)

In [4]:
# Create column for zipcode
df['zipcode'] = 1
for i in range(len(df['address'])):
    df.loc[i, 'zipcode'] = df['address'][i][-20:-15]
df['zipcode'] = df['zipcode'].astype(int)

In [5]:
# Pull list of zipcodes associatied with King County, dataset from https://data.kingcounty.gov/
z = pd.read_csv('data/Zipcodes_for_King_County_and_Surrounding_Area_(Shorelines)___zipcode_shore_area.csv')
z = z[z['COUNTY_NAME'] == 'King County']

In [6]:
# Filter house data DataFrame for zipcodes within County
in_king_county_mask = df['zipcode'].isin(z['ZIPCODE'])
df_king = df[in_king_county_mask]
df_king = df_king.drop_duplicates()

In an effort to find Zip codes where flipping a house returns the most value, we will classify houses based on 'condition' and 'grade'.

According to King County's descriptions of the column names, `condition` is how good the overall condition of the house is from a maintenance perspective. `grade` is the overall grade of the house, related to the construction and design of the house.

BUILDING CONDITION
     	Relative to age and grade. Coded 1-5.

1 = Poor- Worn out. Repair and overhaul needed on painted surfaces, roofing, plumbing, heating and numerous functional inadequacies. Excessive deferred maintenance and abuse, limited value-in-use, approaching abandonment or major reconstruction; reuse or change in occupancy is imminent. Effective age is near the end of the scale regardless of the actual chronological age.

2 = Fair- Badly worn. Much repair needed. Many items need refinishing or overhauling, deferred maintenance obvious, inadequate building utility and systems all shortening the life expectancy and increasing the effective age.

3 = Average- Some evidence of deferred maintenance and normal obsolescence with age in that a few minor repairs are needed, along with some refinishing. All major components still functional and contributing toward an extended life expectancy. Effective age and utility is standard for like properties of its class and usage.

4 = Good- No obvious maintenance required but neither is everything new. Appearance and utility are above the standard and the overall effective age will be lower than the typical property.

5= Very Good- All items well maintained, many having been overhauled and repaired as they have shown signs of wear, increasing the life expectancy and lowering the effective age with little deterioration or obsolescence evident with a high degree of utility.

BUILDING GRADE
     	Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:

1-3 Falls short of minimum building standards. Normally cabin or inferior structure.

4 Generally older, low quality construction. Does not meet code.

5 Low construction costs and workmanship. Small, simple design.

6 Lowest grade currently meeting building code. Low quality materials and simple designs.

7 Average grade of construction and design. Commonly seen in plats and older sub-divisions.

8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

9 Better architectural design with extra interior and exterior design and quality.

10 Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

13 Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

In [7]:
# Create ordinal numerics for both 'condition' and 'grade'
dict = {'Poor':1, 'Fair':2, 'Average':3, 'Good':4, 'Very Good':5}
df_king=df_king.replace({"condition": dict})

df_king['gradeno'] = df_king['grade'].map(lambda x: x[0:2])
df_king['gradeno'] = df_king['gradeno'].astype('int')
df_king = df_king.drop(columns = 'grade')

There will be 3 groups. 0 includes all 'Poor' and 'Fair' condition rows, and well as 'Average' condition where grade is 7 or less. Class 2 includes all 'Very good' condition rows, as well as 'Good' condition where grade is 9 or more. Class 1 includes all else.

In [8]:
# Create class column
def get_class(row):
    """Classify properties into 3 groups. 0 includes all 'Poor' and 'Fair' condition rows, and well as 'Average' condition where grade is 7 or less.
    Class 2 includes all 'Very good' condition rows, as well as 'Good' condition where grade is 9 or more.
    Class 1 includes all else."""
    if row['condition'] <= 2:
        return 0
    elif row['condition'] == 3 and row['gradeno'] <= 7:
        return 0
    elif row['condition'] == 5:
        return 2
    elif row['condition'] == 4 and row['gradeno'] >= 9:
        return 2
    else:
        return 1

df_king['class'] = df_king.apply(get_class, axis=1)

In [9]:
# Find difference between class 2 and class 0 median property values per zip, neglecting zip codes with null values
df_median_price = pd.pivot_table(df_king, values='price', index='zipcode', columns='class', aggfunc='median')
df_median_price = df_median_price.dropna()
df_median_price = df_median_price.reset_index()
df_median_price['diff'] = df_median_price[2] - df_median_price[0]
df_median_price = df_median_price.rename(columns={'zipcode': 'ZIPCODE'})

This is visualized the visualization notebook and in Zip_select_map.py

In [10]:
# View the top 5 in difference in value
df_median_price.sort_values('diff', ascending=False).head()

class,ZIPCODE,0,1,2,diff
24,98039,2515000.0,4300000.0,5100000.0,2585000.0
3,98004,1817000.0,3438000.0,3375000.0,1558000.0
51,98112,950000.0,1355200.0,2275000.0,1325000.0
42,98077,827000.0,1500000.0,1835000.0,1008000.0
46,98105,877500.0,1285000.0,1870000.0,992500.0


We will focus on the top 5 zipcodes.

98039, \$2,585,000, Medina

98004, \$1,558,000, Bellvue

98112, \$1,325,000, Seattle

98077, \$1,008,000, Woodinville

98105, \$992,500, Seattle

In [11]:
# Create DataFrames for our selected Zips.
df1 = df_king[df_king['zipcode']==98039]
df2 = df_king[df_king['zipcode']==98004] 
df3 = df_king[df_king['zipcode']==98112]
df4 = df_king[df_king['zipcode']==98077]
df5 = df_king[df_king['zipcode']==98105] 

# Create a DataFrame of all 5
df15 =  pd.concat([df1,df2,df3,df4,df5])

In [12]:
# Save the Dataframes for later use
df1.to_pickle('data/df1.pkl')
df2.to_pickle('data/df2.pkl')
df3.to_pickle('data/df3.pkl')
df4.to_pickle('data/df4.pkl')
df5.to_pickle('data/df5.pkl')
df15.to_pickle('data/df15.pkl')